In [1]:
import deep_image_matching as dim
import yaml

logger = dim.setup_logger("dim")

params = {
    "dir": "./datasets/belv_20230725",
    "pipeline": "superpoint+lightglue",
    "config_file": "./datasets/belv_20230725/superpoint+lightglue.yaml",
    "strategy": "bruteforce",
    "quality": "highest",
    "tiling": "preselection",
    "skip_reconstruction": False,
    "force": True,
    "camera_options": "./datasets/belv_20230725/cameras.yaml",
    "openmvg": None,
    "verbose": True,
}

# Build configuration
config = dim.Config(params)
config.save()

Deep Image Matching loaded in 3.036 seconds.
2024-04-11 10:28:07 | [WARNING ] datasets/belv_20230725/results_superpoint+lightglue_bruteforce_quality_highest already exists, but the '--force' option is used. Deleting the folder.
Using a custom configuration file: /home/francesco/phd/deep-image-matching/datasets/belv_20230725/superpoint+lightglue.yaml
Config general:
{'camera_options': './datasets/belv_20230725/cameras.yaml',
 'db_path': None,
 'geom_verification': <GeometricVerification.PYDEGENSAC: 1>,
 'graph': True,
 'gv_confidence': 0.99999,
 'gv_threshold': 6,
 'image_dir': PosixPath('datasets/belv_20230725/images'),
 'matching_strategy': 'bruteforce',
 'min_inlier_ratio_per_pair': 0.2,
 'min_inliers_per_pair': 2,
 'min_matches_per_tile': 10,
 'openmvg_conf': None,
 'output_dir': PosixPath('datasets/belv_20230725/results_superpoint+lightglue_bruteforce_quality_highest'),
 'overlap': None,
 'pair_file': PosixPath('datasets/belv_20230725/results_superpoint+lightglue_bruteforce_quality

In [2]:
# Initialize ImageMatcher class
matcher = dim.ImageMatcher(config)

# Run image matching
feature_path, match_path = matcher.run()

# Read camera options
with open(config.general["camera_options"], "r") as file:
    camera_options = yaml.safe_load(file)

# Export in colmap format
database_path = config.general["output_dir"] / "database.db"
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_options=camera_options,
)

2024-04-11 10:28:09 | [DEBUG   ] Matching options: Quality: HIGHEST - Tiling: PRESELECTION
2024-04-11 10:28:09 | [DEBUG   ] Saving directory: datasets/belv_20230725/results_superpoint+lightglue_bruteforce_quality_highest
2024-04-11 10:28:09 | [DEBUG   ] Running inference on device cuda
Loaded SuperPoint model
2024-04-11 10:28:10 | [DEBUG   ] Matching options: Tiling: PRESELECTION
2024-04-11 10:28:10 | [DEBUG   ] Saving directory: datasets/belv_20230725/results_superpoint+lightglue_bruteforce_quality_highest
2024-04-11 10:28:10 | [DEBUG   ] Running inference on device cuda
Loaded SuperPoint model
2024-04-11 10:28:10 | [INFO    ] Running image matching with the following configuration:
2024-04-11 10:28:10 | [INFO    ]   Image folder: datasets/belv_20230725/images
2024-04-11 10:28:10 | [INFO    ]   Output folder: datasets/belv_20230725/results_superpoint+lightglue_bruteforce_quality_highest
2024-04-11 10:28:10 | [INFO    ]   Number of images: 2
2024-04-11 10:28:10 | [INFO    ]   Matching 

  0%|          | 0/2 [00:00<?, ?it/s]

2024-04-11 10:28:49 | [DEBUG   ]   - Extracting features from tile: 0
2024-04-11 10:28:52 | [DEBUG   ]   - Extracting features from tile: 1
2024-04-11 10:28:52 | [DEBUG   ]   - Extracting features from tile: 2
2024-04-11 10:28:52 | [DEBUG   ]   - Extracting features from tile: 3
2024-04-11 10:28:52 | [DEBUG   ]   - Extracting features from tile: 4
2024-04-11 10:28:53 | [DEBUG   ]   - Extracting features from tile: 5
2024-04-11 10:28:53 | [DEBUG   ]   - Extracting features from tile: 6
2024-04-11 10:28:53 | [DEBUG   ]   - Extracting features from tile: 7
2024-04-11 10:28:54 | [DEBUG   ]   - Extracting features from tile: 8
2024-04-11 10:28:54 | [DEBUG   ]   - Extracting features from tile: 9
2024-04-11 10:28:54 | [DEBUG   ]   - Extracting features from tile: 10
2024-04-11 10:28:55 | [DEBUG   ]   - Extracting features from tile: 11
2024-04-11 10:28:55 | [DEBUG   ]   - Extracting features from tile: 12
2024-04-11 10:28:55 | [DEBUG   ]   - Extracting features from tile: 13
2024-04-11 10:28

 50%|█████     | 1/2 [00:08<00:08,  8.67s/it]

2024-04-11 10:28:58 | [DEBUG   ]   - Extracting features from tile: 0
2024-04-11 10:28:58 | [DEBUG   ]   - Extracting features from tile: 1
2024-04-11 10:28:59 | [DEBUG   ]   - Extracting features from tile: 2
2024-04-11 10:28:59 | [DEBUG   ]   - Extracting features from tile: 3
2024-04-11 10:28:59 | [DEBUG   ]   - Extracting features from tile: 4
2024-04-11 10:29:00 | [DEBUG   ]   - Extracting features from tile: 5
2024-04-11 10:29:00 | [DEBUG   ]   - Extracting features from tile: 6
2024-04-11 10:29:00 | [DEBUG   ]   - Extracting features from tile: 7
2024-04-11 10:29:00 | [DEBUG   ]   - Extracting features from tile: 8
2024-04-11 10:29:01 | [DEBUG   ]   - Extracting features from tile: 9
2024-04-11 10:29:01 | [DEBUG   ]   - Extracting features from tile: 10
2024-04-11 10:29:01 | [DEBUG   ]   - Extracting features from tile: 11
2024-04-11 10:29:02 | [DEBUG   ]   - Extracting features from tile: 12
2024-04-11 10:29:02 | [DEBUG   ]   - Extracting features from tile: 13
2024-04-11 10:29

...

/home/francesco/miniforge3/envs/deep-image-matching/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2024-04-11 10:29:23 | [DEBUG   ] [Timer] | [Tile selection] preselection=9.026, Tile selection=0.000, Total execution=9.026
2024-04-11 10:29:23 | [DEBUG   ]  - Matching tile pair (2, 4)
2024-04-11 10:29:23 | [DEBUG   ]      Found 0 matches
2024-04-11 10:29:23 | [DEBUG   ]  - Matching tile pair (3, 4)
2024-04-11 10:29:23 | [DEBUG   ]      Found 571 matches
2024-04-11 10:29:23 | [DEBUG   ]  - Matching tile pair (3, 5)
2024-04-11 10:29:23 | [DEBUG   ]      Found 218 matches
2024-04-11 10:29:23 | [DEBUG   ]  - Matching tile pair (5, 5)
2024-04-11 10:29:24 | [DEBUG   ]      Found 0 matches
2024-04-11 10:29:24 | [DEBUG   ]  - Matching tile pair (5, 8)
2024-04-11 10:29:24 | [DEBUG   ]      Found 0 matches
2024-04-11 10:29:24 | [DEBUG   ]  - Matching tile pair (6, 4)
2024-04-11 10:29:24 | [DEBUG   ]      Found 0 matches
2024-04-11 10:29:24 | [DEBUG   ]  - Matching tile pair (6, 5)
2024-04-11 10:29:24 | [DEBUG   ]      Found 0 matches
2024-04-11 10:29:24 | [DEBUG   ]  - Matching tile pair (6, 9

100%|██████████| 1/1 [00:24<00:00, 24.05s/it]

2024-04-11 10:29:28 | [INFO    ] [Timer] | [matching] generate_pairs=41.395, extract_features=15.481, Match pair=24.053, Total execution=80.979
2024-04-11 10:29:28 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000



100%|██████████| 2/2 [00:00<00:00, 403.45it/s]
1it [00:00, 1136.98it/s]
1it [00:00, 1162.18it/s]


In [8]:
# Set camera options

import numpy as np

cameras = {
    1: {
        "model": 4,  # 4 is OPENCV model
        "width": 6000,
        "height": 4000,
        "params": [
            9.26789262766209504e03,
            9.26789262766209504e03,
            3.05349107994520591e03,
            1.94835654532114540e03,
            -8.07042713029020586e-02,
            9.46617629940955385e-02,
            3.31782983128223608e-04,
            -4.32106111976037410e-04,
        ],
    },
    2: {
        "model": 4,
        "width": 6000,
        "height": 4000,
        "params": [
            6.62174345720628298e03,
            6.62174345720628298e03,
            3.01324420057086490e03,
            1.94347461466223308e03,
            -9.41830394356213407e-02,
            8.55303528514532035e-02,
            1.68948638308769863e-04,
            -8.74637609310216697e-04,
        ],
    },
}
with dim.utils.COLMAPDatabase(database_path) as db:
    for id, cam in cameras.items():
        db.update_camera(
            camera_id=id,
            model=cam["model"],
            width=cam["width"],
            height=cam["height"],
            params=cam["params"],
        )

In [9]:
# import pycolmap

# print(pycolmap.IncrementalPipelineOptions().summary())

In [10]:
# Run reconstruction
opt = dict(
    triangulation=dict(
        ignore_two_view_tracks=False,
        min_angle=0.5,
    ),
    mapper=dict(filter_min_tri_angle=0.5, filter_max_reproj_error=5.0),
)
refine_intrinsics = False
verbose = False

model = dim.reconstruction.pycolmap_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=refine_intrinsics,
    options=opt,
    verbose=verbose,
)

2024-04-10 12:25:19 | [INFO    ] Running 3D reconstruction...


I20240410 12:25:19.521893 843404 misc.cc:198] 
Loading database
I20240410 12:25:19.529183 843404 database_cache.cc:54] Loading cameras...
I20240410 12:25:19.529922 843404 database_cache.cc:64]  2 in 0.001s
I20240410 12:25:19.529970 843404 database_cache.cc:72] Loading matches...
I20240410 12:25:19.530303 843404 database_cache.cc:78]  1 in 0.000s
I20240410 12:25:19.530313 843404 database_cache.cc:94] Loading images...
I20240410 12:25:19.534248 843404 database_cache.cc:143]  2 in 0.004s (connected 2)
I20240410 12:25:19.534322 843404 database_cache.cc:154] Building correspondence graph...
W20240410 12:25:19.534687 843404 correspondence_graph.cc:156] Duplicate correspondence between point2D_idx=56963 in image_id=1 and point2D_idx=20004 in image_id=2
I20240410 12:25:19.536156 843404 database_cache.cc:190]  in 0.002s (ignored 0)
I20240410 12:25:19.536219 843404 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240410 12:25:19.537696 843404 misc.cc:198] 
Finding good initial image pair
I20240410 

2024-04-10 12:25:19 | [INFO    ] Reconstructed 1 model(s).
2024-04-10 12:25:19 | [INFO    ] Largest model is #0 with 2 images.
2024-04-10 12:25:19 | [INFO    ] Exporting model...
